In [1]:
from pathlib import Path

import os
from dotenv import load_dotenv
import geopandas as gpd

In [2]:
load_dotenv()

True

In [3]:
BASE_DIR = Path(os.getenv("BASE_DIR"))

In [ ]:
from pathlib import Path
import geopandas as gpd
import pandas as pd
import numpy as np

base_dir = Path(BASE_DIR / "Train_Final2")
pred_dir = base_dir / "ml_runs_pu_ensemble" / "pos1_bg2"

# Load prediction
# pred_df = pd.read_parquet(pred_dir / "pred_mean_std.parquet")
pred_df = pd.read_parquet(BASE_DIR / "Train_Final2/new_data_pred_ensemble.parquet")



In [2]:
pred_df.columns

Index(['dem_mean', 'dem_std', 'dem_p90', 'slope_mean', 'slope_std',
       'slope_p90', 'aspect_sin_mean', 'aspect_cos_mean', 'roughness_mean',
       'roughness_std', 'roughness_p90', 'curv_profile_mean',
       'curv_profile_std', 'curv_profile_p90', 'tmi_mean', 'tmi_std',
       'tmi_p90', 'rtp_mean', 'rtp_std', 'rtp_p90', '1vd_mean', '1vd_std',
       '1vd_p90', 'dist_mag_line_m', 'mag_nearest_amp', 'mag_len_5km',
       'mag_amp_max_5km', 'dist_grav_line_m', 'grav_nearest_amp',
       'grav_len_5km', 'grav_amp_max_5km', 'r_CU_PPM_mean', 'r_AU_PPM_mean',
       'r_AG_PPM_mean', 'r_AS_PPM_mean', 'r_MO_PPM_mean', 'r_PB_PPM_mean',
       'r_ZN_PPM_mean', 'r_CO_PPM_mean', 'r_NI_PPM_mean', 'r_SB_PPM_mean',
       'r_W_PPM_mean', 'r_BI_PPM_mean', 'r_SN_PPM_mean', 'r_rock_count',
       's_CU_PPM_median_pt_mean', 's_CU_PPM_log1p_median_pt_mean',
       's_AS_PPM_median_pt_mean', 's_AU_PPB_median_pt_mean',
       's_MO_PPM_median_pt_mean', 's_SB_PPM_median_pt_mean',
       's_ZN_PPM_median

In [3]:
pred_df = pred_df[["pred_mean",	"pred_std"]]

In [4]:
pred_df

,pred_mean,pred_std
0,0.000910,0.0
1,0.001458,0.0
2,0.002419,0.0
3,0.001710,0.0
4,0.000839,0.0
...,...,...
6524347,0.000031,0.0
6524348,0.000031,0.0
6524349,0.000031,0.0
6524350,0.000031,0.0


In [ ]:
gpd_grid_ml = gpd.read_file(BASE_DIR / "Main output/geometry_poly.gpkg")

In [6]:
gpd_grid_ml

,y,geometry
0,0,"POLYGON ((500550.886 7093412.256, 500550.886 7..."
1,0,"POLYGON ((500550.886 7093912.256, 500550.886 7..."
2,0,"POLYGON ((500550.886 7094412.256, 500550.886 7..."
3,0,"POLYGON ((500550.886 7094912.256, 500550.886 7..."
4,0,"POLYGON ((500550.886 7095412.256, 500550.886 7..."
...,...,...
6524347,0,"POLYGON ((1464050.886 8782912.256, 1464050.886..."
6524348,0,"POLYGON ((1464050.886 8783412.256, 1464050.886..."
6524349,0,"POLYGON ((1464050.886 8783912.256, 1464050.886..."
6524350,0,"POLYGON ((1464050.886 8784412.256, 1464050.886..."


In [ ]:

gpd_grid_pred = gpd_grid_ml.join(pred_df)


gpd_grid_pred["pred_risk_adj"] = (
    gpd_grid_pred["pred_mean"] - gpd_grid_pred["pred_std"]
)


In [8]:
print(type(gpd_grid_pred), gpd_grid_pred.geometry.name)

<class 'geopandas.geodataframe.GeoDataFrame'> geometry


In [9]:
gpd_grid_pred

,y,geometry,pred_mean,pred_std,pred_risk_adj
0,0,"POLYGON ((500550.886 7093412.256, 500550.886 7...",0.000910,0.0,0.000910
1,0,"POLYGON ((500550.886 7093912.256, 500550.886 7...",0.001458,0.0,0.001458
2,0,"POLYGON ((500550.886 7094412.256, 500550.886 7...",0.002419,0.0,0.002419
3,0,"POLYGON ((500550.886 7094912.256, 500550.886 7...",0.001710,0.0,0.001710
4,0,"POLYGON ((500550.886 7095412.256, 500550.886 7...",0.000839,0.0,0.000839
...,...,...,...,...,...
6524347,0,"POLYGON ((1464050.886 8782912.256, 1464050.886...",0.000031,0.0,0.000031
6524348,0,"POLYGON ((1464050.886 8783412.256, 1464050.886...",0.000031,0.0,0.000031
6524349,0,"POLYGON ((1464050.886 8783912.256, 1464050.886...",0.000031,0.0,0.000031
6524350,0,"POLYGON ((1464050.886 8784412.256, 1464050.886...",0.000031,0.0,0.000031


In [10]:
out_map = base_dir / "cu_prospectivity_maps_new.gpkg"

gpd_grid_pred.to_file(
    out_map,
    layer="grid_500m_pred",
    driver="GPKG"
)

print("Saved:", out_map)


Saved: C:\Users\Phong\Desktop\GIS\Project 2\Train_Final2\cu_prospectivity_maps_new.gpkg


In [ ]:

mask_unlabelled = gpd_grid_pred["y"] == 0

df_unlab = gpd_grid_pred.loc[mask_unlabelled].copy()

thr_1pct = np.quantile(df_unlab["pred_mean"], 0.99)
thr_5pct  = np.quantile(df_unlab["pred_mean"], 0.95)

df_unlab["top_class"] = "other"
df_unlab.loc[df_unlab["pred_mean"] >= thr_5pct, "top_class"] = "top_5pct"
df_unlab.loc[df_unlab["pred_mean"] >= thr_1pct, "top_class"] = "top_1pct"

df_unlab["top_class"].value_counts()


top_class
other       6110117
top_5pct     257268
top_1pct      64318
Name: count, dtype: int64

In [13]:
thr_1pct

np.float32(0.9768792)

In [14]:
thr_5pct 

np.float32(0.81106776)

In [ ]:
# Only get top 1% và 5%
targets = df_unlab[df_unlab["top_class"].isin(["top_1pct", "top_5pct"])]

# Dissolve by  class 
targets_poly = (
    targets
    .dissolve(by="top_class", as_index=False)
)


targets_poly["area_km2"] = targets_poly.geometry.area / 1e6


In [17]:
out_targets = base_dir / "cu_exploration_targets_new.gpkg"

targets_poly.to_file(
    out_targets,
    layer="cu_targets",
    driver="GPKG"
)

print("Saved targets:", out_targets)

Saved targets: C:\Users\Phong\Desktop\GIS\Project 2\Train_Final2\cu_exploration_targets_new.gpkg


In [18]:
targets_poly

,top_class,geometry,y,pred_mean,pred_std,pred_risk_adj,area_km2
0,top_1pct,"MULTIPOLYGON (((505050.886 7126912.256, 505050...",0,0.986043,0.0,0.986043,16079.5
1,top_5pct,"MULTIPOLYGON (((503050.886 7124912.256, 502550...",0,0.856248,0.0,0.856248,64317.0
